In [3]:
import requests
import json
import pandas as pd
from datetime import datetime

webpage = requests.get("https://wusa.ca/events/")
jsonscript =str(webpage.content)
isolatedinformation=jsonscript.split('<script type="application/ld+json">')[1].split("</script>")[0][4:-4].encode("utf16", errors="surrogatepass").decode("utf16").encode().decode('unicode_escape')

json = json.loads(isolatedinformation.replace('&lt;p&gt;',"").replace("[&hellip;]&lt;/p&gt;\\\\n",""))

postsDf ={}
postscolumns = ['account','date','caption',"display_photo",'event_details']
postsDf = pd.DataFrame(columns = postscolumns)

[{'@context': 'http://schema.org', '@type': 'Event', 'name': 'First Year Fair', 'description': '📢 Hey WUSA Members! If this is your first time at Waterloo, welcome! And for those ', 'image': 'https://8d3cdb9a.rocketcdn.me/wp-content/uploads/2024/07/FirstYearFair_Web.png', 'url': 'https://wusa.ca/event/first-year-fair/', 'eventAttendanceMode': 'https://schema.org/OfflineEventAttendanceMode', 'eventStatus': 'https://schema.org/EventScheduled', 'startDate': '2024-09-05T11:00:00-04:00', 'endDate': '2024-09-05T14:00:00-04:00', 'location': {'@type': 'Place', 'name': 'SLC Green Space', 'description': '', 'url': 'https://wusa.ca/venue/slc-green-space/', 'address': {'@type': 'PostalAddress', 'streetAddress': '200 University Ave W, SLC', 'addressLocality': 'Waterloo', 'addressRegion': 'Ontario', 'postalCode': 'N2L 3G1', 'addressCountry': 'Canada'}, 'geo': {'@type': 'GeoCoordinates', 'latitude': 40.7607793, 'longitude': -111.8910474}, 'telephone': '2262436643', 'sameAs': ''}, 'organizer': {'@type

In [4]:
import time

time_now = int(time.time())
for event in json:
    if int(datetime.fromisoformat(event["startDate"]).timestamp()) >= time_now:
        try: 
            location = event["location"]["address"]["streetAddress"]
        except Exception as err:
            print(str(err))
            location = None
        
        new_row = pd.DataFrame({
                "account": ["WUSA"],
                "date": [time_now],
                "caption": str(event["description"])+" [For More Information, Click View Post] ",
                "display_photo": event["image"],
                "url": [event['url']],
                "likes": [0],
                "event_details": [{
                    "is_event": True,
                    "event_name": event["name"],
                    "event_description": str(event["description"])+" ... ",
                    "categories": ["Social"],
                    "start_time": int(datetime.fromisoformat(event["startDate"]).timestamp()),
                    "end_time": int(datetime.fromisoformat(event["endDate"]).timestamp()),
                    "location": location,
                }]
        })

        postsDf = pd.concat([postsDf, new_row])

print(postsDf)
postsDf.to_csv(path_or_buf="WUSA events")

'location'
'streetAddress'
'streetAddress'
'streetAddress'
  account        date                                            caption  \
0    WUSA  1725310142  📢 Hey WUSA Members! If this is your first time...   
0    WUSA  1725310142  CIBC x WUSA x UWaterloo presents Dillon Franci...   
0    WUSA  1725310142  Welcome Week 🍂 Get ready to FALL back into fun...   
0    WUSA  1725310142  Rise and shine, Warriors! WUSA’s inviting you ...   
0    WUSA  1725310142  Esports Gaming Lounge Game on, Warriors! Brace...   
0    WUSA  1725310142  😍 A Welcome Week favourite is back! Are you re...   
0    WUSA  1725310142  Step right up! The WUSA Carnival is back for T...   
0    WUSA  1725310142  To give you a warm welcome back to campus, we ...   
0    WUSA  1725310142  Step right up! The WUSA Carnival is back for T...   
0    WUSA  1725310142  Don\'t miss out on the ultimate bowling experi...   
0    WUSA  1725310142  Esports Gaming Lounge Game on, Warriors! Brace...   
0    WUSA  1725310142  The Ca

In [5]:
import os
from dotenv import load_dotenv
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
import pandas as pd

load_dotenv()
uri = os.getenv('DATABASE_URI')
client = MongoClient(uri, server_api=ServerApi('1'))
db = client['Instagram']
collection = db["Events"]

# Remove all documents with these IDs
result = collection.delete_many({'account': "WUSA"})

# Print the number of documents deleted
print(f"Documents removed: {result.deleted_count}")


Documents removed: 0


In [6]:
from typing import List
from together import Together
import time

together_api_key = os.getenv('TOGETHER_API')


#code for embedding
embedding_model_string = 'WhereIsAI/UAE-Large-V1' # model API string from Together.

def generate_embedding(input_texts: List[str], model_api_string: str) -> List[List[float]]:
  together_client = Together(api_key=together_api_key)
  outputs = together_client.embeddings.create(
      input=input_texts,
      model=model_api_string,
  )
  return [x.embedding for x in outputs.data]

insertObjectIds = []

for index, row in postsDf.iterrows():
    info = f'"id": "{int(index)}"|* "account": "{row["account"]}"|* "date": "{row["date"]}"|* "caption": "{row["caption"]}"|*'
    embbededtext = ',\n'.join(x for x in info.replace('\n','\\n').split('|*')) 

    row_dict = {}
    for column in postsDf.columns.tolist():
        row_dict[column] = row[column]
    row_dict["embedded"] = generate_embedding([embbededtext], embedding_model_string)  
    result = collection.insert_one(row_dict)
    print(f"Inserted document ID: {result.inserted_id}")
    insertObjectIds.append(result.inserted_id)
    time.sleep(1)


Inserted document ID: 66d624ca386fda1bc506a567
Inserted document ID: 66d624cc386fda1bc506a568
Inserted document ID: 66d624cd386fda1bc506a569
Inserted document ID: 66d624ce386fda1bc506a56a
Inserted document ID: 66d624d0386fda1bc506a56b
Inserted document ID: 66d624d1386fda1bc506a56c
Inserted document ID: 66d624d2386fda1bc506a56d
Inserted document ID: 66d624d3386fda1bc506a56e
Inserted document ID: 66d624d5386fda1bc506a56f
Inserted document ID: 66d624d6386fda1bc506a570
Inserted document ID: 66d624d8386fda1bc506a571
Inserted document ID: 66d624d9386fda1bc506a572
Inserted document ID: 66d624da386fda1bc506a573
Inserted document ID: 66d624dc386fda1bc506a574
Inserted document ID: 66d624dd386fda1bc506a575
Inserted document ID: 66d624de386fda1bc506a576
Inserted document ID: 66d624e0386fda1bc506a577
Inserted document ID: 66d624e1386fda1bc506a578
Inserted document ID: 66d624e2386fda1bc506a579
Inserted document ID: 66d624e3386fda1bc506a57a
Inserted document ID: 66d624e5386fda1bc506a57b
